The example code below shows how to instantiate a TouchPlotter object from the plotter module and use it to perform real-time plotting of touch data in an IPython notebook.

By default, the plot is generated directly below the code cell and provides a view of finger positions. These settings can be adjusted by passing in parameters when instantiating the TouchPlotter object. For example, *plotter = TouchPlotter(mode='jupyterlab')* generates the plot in a separate tab in JupyterLab instead of directly below the code cell. With TouchPlotter, two of the parameters are mandatory, namely max_x and max_y. The values of these two parameters represent the maximum X and Y coordinate values of touch data and can be extracted from the Application Info packet. The available parameters of TouchPlotter are listed below.

**mode** (default = 'inline')<br/>
&emsp;'inline' - Generate plot directly below code cell<br/>
&emsp;'jupyterlab' - Generate plot in separate tab in JupyterLab<br/>
&emsp;'external' - Generate hyperlink for displaying plot in separate browser tab

**port** (default = 8050)<br/>
&emsp;This parameter is for setting the port through which this plot is served from the JupyterDash server running on the DSDK.

**view** (default = 'position')<br/>
&emsp;'position' - Display finger positions in plot<br/>
&emsp;'trace' - Display finger traces in plot<br/>

**max_x** (<span style='color:#007dc3 '>*mandatory*</span>)<br/>
&emsp;This parameter represents the maximum X coordinate value of touch data.

**max_y** (<span style='color:#007dc3 '>*mandatory*</span>)<br/>
&emsp;This parameter represents the maximum Y coordinate value of touch data.

**plot_scale** (default = 1)<br/>
&emsp;This parameter is a scale factor for determining the size of the plot.

TouchPlotter relies on the availability of the private config JSON file for the firmware on the target TouchComm device for its plotting functionality. To make the private config JSON file available to TouchPlotter, it can be placed inside the Packrat directory in the file browser in the left sidebar. For example, for PR1234567, the location for the private config JSON file is /Packrat/1234567/config_private.json.

To terminate the plotting session, go to the Kernel main menu and do Interrupt Kernel and Restart Kernel. This triggers the KeyboardInterrupt exception shown at the bottom of the example code and allows the plotter module to be used in other IPython notebooks for real-time plotting.
<br/><br/>
***
**Note**: The plotter module makes use of JupyterDash, which by default is served from the DSDK through port 8050. This means that in order for plotting to work, the "adb forward" command (e.g. "adb forward tcp:8050 tcp:8050" for port 8050) needs to be issued first from the host computer this browser is running on.<br/>
**Note**: It is recommended to use either Firefox or Sarafi for running this real-time plotting feature from within an IPython notebook.
***

In [ ]:
import sys
import time
import import_ipynb
from plotter import TouchPlotter
from touchcomm import TouchComm

fps = 60
period = 1/fps

with TouchComm.make('report_streamer', streaming=False) as tc:
    tc.reset()
    app_info = tc.getAppInfo()
    max_x = app_info['maxX']
    max_y = app_info['maxY']
    tc.enableReport(17)

    try:
        plotter = TouchPlotter(max_x=max_x, max_y=max_y)
        plotter.run_plot()
        while True:
            plotter.update_plot(tc.getReport())
            time.sleep(period)
    except KeyboardInterrupt:
        pass
    except Exception as e:
        print(e)
    finally:
        if plotter is not None:
            plotter.stop_plot()
